## Reading DF

In [2]:

from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext("local[*]")
spark = SparkSession.builder.getOrCreate()
import pandas as pd  


23/02/22 21:53:56 WARN Utils: Your hostname, ubuntu-hadoop resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/22 21:53:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/22 21:53:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark = SparkSession.builder.getOrCreate()
from  pyspark.sql.functions import split

## reading the data 

In [4]:
file_path_testing ="../data/test.txt"
file_path_training= "../data/train.txt"
file_path_val= "../data/val.txt"


def readFile(file_path):
    df_row = spark.read.csv(file_path)
    # print("original")
    # print(df_row.show(4,False))
    df= df_row.withColumn("text", split(df_row['_c0'], ';').getItem(0))\
            .withColumn('emotion', split(df_row['_c0'], ';').getItem(1))
    df1= df.select("text", "emotion")
    return df1

df_training = readFile(file_path= file_path_training)
df_testing = readFile(file_path= file_path_training)
df_val = readFile(file_path= file_path_val)
print(df_val.show(4,False))

+----------------------------------------------------------------------+-------+
|text                                                                  |emotion|
+----------------------------------------------------------------------+-------+
|im feeling quite sad and sorry for myself but ill snap out of it soon |sadness|
|i feel like i am still looking at a blank canvas blank pieces of paper|sadness|
|i feel like a faithful servant                                        |love   |
|i am just feeling cranky and blue                                     |anger  |
+----------------------------------------------------------------------+-------+
only showing top 4 rows

None


## data prep

In [5]:
import pandas as pd 
print("training size ",df_training.toPandas().shape , "data points") 
print("testing size ",df_testing.toPandas().shape, "data points")
print("validation size ",df_val.toPandas().shape, "data points") 

training size  (16000, 2) data points


testing size  (16000, 2) data points
validation size  (2000, 2) data points


we see that we have unbalance data we have a lot of joy and sadness label and very little surprise and fear

In [9]:
print("training", df_training.select("emotion").distinct().show())
print("training class count", df_training.groupBy("emotion").count().show())

+--------+
| emotion|
+--------+
|     joy|
|    love|
|   anger|
|    fear|
|surprise|
| sadness|
+--------+

training None
+--------+-----+
| emotion|count|
+--------+-----+
|     joy| 5362|
|    love| 1304|
|   anger| 2159|
|    fear| 1937|
|surprise|  572|
| sadness| 4666|
+--------+-----+

training class count None


# convert labels to the numbers 

In [10]:
from pyspark.ml.feature import StringIndexer # to convert class labels to numeric
from pyspark.sql.functions import col
# Convert sentiment to numbers  joy =0 love =1 
#converting class labels
string_indexer = StringIndexer(inputCol="emotion",outputCol="class")

# fit the StringIndexer to the DataFrame and transform the DataFrame to add the 'fruits_numeric' column
df_class = string_indexer.fit(df_training).transform(df_training)\
.withColumn('class', col('class').cast("integer"))
df_class.show(5)

+--------------------+-------+-----+
|                text|emotion|class|
+--------------------+-------+-----+
|i didnt feel humi...|sadness|    1|
|i can go from fee...|sadness|    1|
|im grabbing a min...|  anger|    2|
|i am ever feeling...|   love|    4|
|i am feeling grouchy|  anger|    2|
+--------------------+-------+-----+
only showing top 5 rows



In [11]:
# fit the StringIndexer to the DataFrame and transform the DataFrame to add the 'class_numeric' column
df_class = string_indexer.fit(df_training).transform(df_training).withColumn('class', col('class').cast("integer"))
print("training class count", df_class.groupBy("emotion","class").count()
.show(truncate=False))


+--------+-----+-----+
|emotion |class|count|
+--------+-----+-----+
|sadness |1    |4666 |
|love    |4    |1304 |
|surprise|5    |572  |
|joy     |0    |5362 |
|fear    |3    |1937 |
|anger   |2    |2159 |
+--------+-----+-----+

training class count None


In [19]:
# fit the StringIndexer to the DataFrame and transform the DataFrame to add the 'class_numeric' column
df_training_class = string_indexer.fit(df_training).transform(df_training).withColumn('class', col('class').cast("integer"))
df_testing_class = string_indexer.fit(df_testing).transform(df_testing).withColumn('class', col('class').cast("integer"))
df_val_class = string_indexer.fit(df_val).transform(df_val).withColumn('class', col('class').cast("integer"))

df_label1 = df_training_class.groupBy("emotion","class").count().select("emotion", "class").show()
df_label2 = df_testing_class.groupBy("emotion","class").count().select("emotion", "class").show()
df_label3 = df_val_class.groupBy("emotion","class").count().select("emotion", "class").show(truncate=False)



+--------+-----+
| emotion|class|
+--------+-----+
| sadness|    1|
|    love|    4|
|surprise|    5|
|     joy|    0|
|    fear|    3|
|   anger|    2|
+--------+-----+

+--------+-----+
| emotion|class|
+--------+-----+
| sadness|    1|
|    love|    4|
|surprise|    5|
|     joy|    0|
|    fear|    3|
|   anger|    2|
+--------+-----+

+--------+-----+
|emotion |class|
+--------+-----+
|sadness |1    |
|love    |4    |
|surprise|5    |
|joy     |0    |
|fear    |3    |
|anger   |2    |
+--------+-----+

None


In [23]:
df_val_class.groupBy("emotion","class").count().select("emotion", "class").show(truncate=False)

+--------+-----+
|emotion |class|
+--------+-----+
|sadness |1    |
|love    |4    |
|surprise|5    |
|joy     |0    |
|fear    |3    |
|anger   |2    |
+--------+-----+



cleaning involves removing punitions and symbols 

In [26]:
from pyspark.sql import functions as F
# cleaning 
def clean(df):
    # Remove html tags from text
    df = df.withColumn("text_c", F.regexp_replace(F.col("text"), r'<[^>]+>', ""));
    # Remove non-letters
    df = df.withColumn("text_c", F.regexp_replace("text_c", r"[^a-zA-Z ]", ""))
    # Remove words 1, 2 char
    df = df.withColumn("text_c", F.regexp_replace("text_c", r"\b\w{1,2}\b", ""))
    return df

